### Reporting days spent on development activity category in given month using Story Points

In [244]:
from jira import JIRA
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from dateutil.rrule import rrule, MONTHLY
from datetime import datetime
import calendar
import datetime

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

jira_url = 'https://kainos-evolve.atlassian.net'
jira = JIRA(jira_url)

In [245]:
issues = pd.DataFrame()
issues['key'] = ''
issues['category'] = ''
issues['type'] = ''
issues['team'] = ''
issues['SP'] = ''
issues['bugEffort'] = ''
issues['status'] = ''
issues['summary'] = ''
issues['month'] = ''

In [246]:
year = 2018

In [247]:
# download data for each month of a year defined previously
for i in range(1,13):
    lastDay = calendar.monthrange(year,i)[1]
    #don't query Jira for months in the future
    now = datetime.datetime.now()
    if i > now.month: break
    
    jql = 'project = vxt and status = Completed AND status changed to Completed during ({}-{}-1, {}-{}-{}) \
 AND type in (bug, "External Bug", story)'.format(year, i, year, i, lastDay)

    issuesRaw = jira.search_issues(jql, maxResults = False)
    jql
    for issue in issuesRaw:

        try:
            bugEffort = issue.fields.customfield_14703
        except AttributeError:
            bugEffort = np.nan

        try:
            team = str(issue.fields.customfield_14200.value)
        except AttributeError:
            team = np.nan

        #category is stored in custom field
        try:
            category = str(issue.fields.customfield_14709.value)
        except AttributeError:
            category = 'Not set'

        issues = issues.append(
            {
             'key': issue.key,
             'type': issue.fields.issuetype.name,
             'status': issue.fields.status.name,
             'category': issue.fields.customfield_14709,
             'bugEffort': bugEffort,
             'SP' : issue.fields.customfield_10005,
             'summary': issue.fields.summary,
             'team': team,
             'month': i
            }, ignore_index=True)



'project = vxt and status = Completed AND status changed to Completed during (2018-1-1, 2018-1-31)  AND type in (bug, "External Bug", story)'

'project = vxt and status = Completed AND status changed to Completed during (2018-2-1, 2018-2-28)  AND type in (bug, "External Bug", story)'

In [248]:
#use bug effort if SP is empty for bug type issues
issues.loc[(issues['type'] == 'Bug') & (issues['SP'].isnull()), ["SP"]] = issues.bugEffort

#filter teams
issues = issues.loc[(issues['team'].isin(["Gdansk Team 1", "Gdansk Team 2", "Belfast Team"]))]


In [249]:
issues['SP']=issues['SP'].fillna(0)

'not estimated issues or issues with zero SP'
notEstimated = issues.loc[(issues['type'] == 'Story') & (issues['SP'] == 0)]
notEstimated


#issues['category']=issues['category'].fillna('Not set')
issues['category'] = issues.category.astype(str)



'not estimated issues or issues with zero SP'

,key,category,type,team,SP,bugEffort,status,summary,month
34,VXT-3810,IC Platform - NFRs,Story,Gdansk Team 1,0.0,NaN,Completed,Reimplementation of password reset endpoint,1
40,VXT-3716,IC Platform - Core Services,Story,Gdansk Team 1,0.0,NaN,Completed,Cluster testing of Keycloak,1
43,VXT-3685,Care Pathway Framework - Controls,Story,Gdansk Team 2,0.0,NaN,Completed,update doc and ui tests,1
63,VXT-2593,IC Platform - NFRs,Story,Gdansk Team 1,0.0,NaN,Completed,HTML5 CORS Misconfiguration,1
64,VXT-2435,IC Platform - Core Services,Story,Gdansk Team 1,0.0,NaN,Completed,Implement consul-template for BPM service,1
65,VXT-2428,IC Platform - Core Services,Story,Gdansk Team 1,0.0,NaN,Completed,Implement consul-template for Integration-webs...,1
66,VXT-2427,IC Platform - Core Services,Story,Gdansk Team 1,0.0,NaN,Completed,Implement consul-template for Integration-engi...,1


In [250]:
#group by by month and category and sum SP
issuesByCategory = issues.groupby(['month', 'category']).agg({'SP':'sum'})

#add column with SP % per month
issuesByCategory['SPPerc'] = issuesByCategory.groupby(level=[0]).apply(lambda x: x / float(x.sum()))
#reindexing needed to relate month to each row 
issuesByCategory = issuesByCategory.reset_index()

#days from Kimble used for calculations
monthDevDays = [100, 100]

#first add days from monthDevDays for specified month
issuesByCategory['Days'] = issuesByCategory['month'].apply(lambda x: monthDevDays[x-1])
#then multiply by % of month spent on category
issuesByCategory['Days'] = issuesByCategory['Days'] * issuesByCategory['SPPerc']
#round values
issuesByCategory = issuesByCategory.round({'SPPerc': 2, 'Days':2})

#
issuesByCategory


,month,category,SP,SPPerc,Days
0,1,Care Pathway Framework - Apps,2.0,0.04,4.04
1,1,Care Pathway Framework - Controls,24.5,0.49,49.49
2,1,IC Platform - Core Services,3.0,0.06,6.06
3,1,IC Platform - NFRs,15.0,0.30,30.30
4,1,Integration & Connector Framework,5.0,0.10,10.10
5,2,Care Pathway Framework - Apps,3.0,0.08,7.69
6,2,Care Pathway Framework - Controls,14.5,0.37,37.18
7,2,IC Platform - Core Services,16.0,0.41,41.03
8,2,IC Platform - NFRs,2.0,0.05,5.13
9,2,Integration & Connector Framework,3.5,0.09,8.97
